In [1]:
%%writefile loa_gpu.cu

#include <bits/stdc++.h>
using namespace std;

#define POP     256       // GPU multiple-of-32 friendly
#define DIM     5
#define MAX_IT  8000
#define LB     -100.0f
#define UB      100.0f

// --------------------------------------
// GPU random generator (XORSHIFT)
// --------------------------------------
__device__ float randf(unsigned int &state){
    state ^= state << 13;
    state ^= state >> 17;
    state ^= state << 5;
    return (state % 10000) / 10000.0f;
}

// --------------------------------------
// Sphere Fitness
// --------------------------------------
__device__ float sphere(float *x) {
    float sum=0;
    for(int i=0;i<DIM;i++) sum += x[i]*x[i];
    return sum;
}

// --------------------------------------
// CUDA Kernel → LOA iteration update
// --------------------------------------
__global__ void LOA_iter(float *pop, float *fitness, float *bestVal, float *bestSol, int iter){

    __shared__ float s_pop[POP][DIM];   // Shared population cache
    __shared__ float s_fit[POP];        // Shared fitness cache

    int i = threadIdx.x;  // each bird = each CUDA thread
    int gid = i * DIM;

    // Copy into shared mem (fast)
    for(int d=0;d<DIM;d++) s_pop[i][d] = pop[gid+d];
    s_fit[i] = fitness[i];
    __syncthreads();

    unsigned int rng = clock64() ^ (i*7919);

    // Find a better random candidate
    int better=-1;
    for(int j=0;j<POP;j++){
        if(s_fit[j] < s_fit[i]) better=j;
    }

    float newPos[DIM];
    if(randf(rng) < 0.5f && better!=-1){
        // ESCAPE
        for(int d=0;d<DIM;d++){
            float r=randf(rng);
            int I=(rng%2)+1;
            newPos[d]=s_pop[i][d] + r*(s_pop[better][d]-I*s_pop[i][d]);
        }
    } else {
        // HIDE
        for(int d=0;d<DIM;d++){
            float r=randf(rng);
            newPos[d]=s_pop[i][d] + (1-2*r)*(UB-LB)/(float)iter;
        }
    }

    // Boundary
    for(int d=0;d<DIM;d++){
        if(newPos[d]<LB)newPos[d]=LB;
        if(newPos[d]>UB)newPos[d]=UB;
    }

    float f=sphere(newPos);

    // Accept if better
    if(f < s_fit[i]){
        for(int d=0;d<DIM;d++) s_pop[i][d]=newPos[d];
        s_fit[i]=f;
    }
    __syncthreads();

    // Global best update (critical section)
    if(i==0){
        int bestIdx=0;
        for(int k=1;k<POP;k++) if(s_fit[k]<s_fit[bestIdx]) bestIdx=k;

        float v=s_fit[bestIdx];
        if(v < *bestVal){
            *bestVal = v;
            for(int d=0;d<DIM;d++) bestSol[d]=s_pop[bestIdx][d];
        }
    }

    // Write back to global mem
    for(int d=0;d<DIM;d++) pop[gid+d]=s_pop[i][d];
    fitness[i]=s_fit[i];
}

// --------------------------------------
// CPU main
// --------------------------------------
int main(){
    float *d_pop, *d_fit, *bestSol, *bestFit;
    cudaMalloc(&d_pop, POP*DIM*sizeof(float));
    cudaMalloc(&d_fit, POP*sizeof(float));
    cudaMalloc(&bestSol, DIM*sizeof(float));
    cudaMalloc(&bestFit, sizeof(float));

    vector<float> h_pop(POP*DIM);
    vector<float> h_fit(POP);

    // Random initialization
    for(int i=0;i<POP;i++){
        for(int j=0;j<DIM;j++){
            h_pop[i*DIM+j]=((float)rand()/RAND_MAX)*(UB-LB)+LB;
        }
        float s=0; for(int j=0;j<DIM;j++) s+=h_pop[i*DIM+j]*h_pop[i*DIM+j];
        h_fit[i]=s;
    }

    cudaMemcpy(d_pop,h_pop.data(),POP*DIM*sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_fit,h_fit.data(),POP*sizeof(float),cudaMemcpyHostToDevice);

    float inf=1e30;
    cudaMemcpy(bestFit,&inf,sizeof(float),cudaMemcpyHostToDevice);

    // ==========================================================
    // ⏳ GPU TIME START
    // ==========================================================
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start); // start timing
    // ==========================================================

    // ------------------- MAIN GPU LOOP -------------------
    for(int t=1;t<=MAX_IT;t++){
        LOA_iter<<<1,POP>>>(d_pop,d_fit,bestFit,bestSol,t);
        cudaDeviceSynchronize();

        if(t%1000==0){
            float bf;
            cudaMemcpy(&bf,bestFit,sizeof(float),cudaMemcpyDeviceToHost);
            cout<<"Iteration "<<t<<" | best = "<<bf<<"\n";
        }
    }

    // ==========================================================
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    // ==========================================================

    float finalFit; vector<float> sol(DIM);
    cudaMemcpy(&finalFit,bestFit,sizeof(float),cudaMemcpyDeviceToHost);
    cudaMemcpy(sol.data(),bestSol,DIM*sizeof(float),cudaMemcpyDeviceToHost);

    cout<<"\n🔥 FINAL BEST = "<<finalFit<<"\n";
    cout<<"Best Solution = "; for(float v:sol) cout<<v<<" ";
    cout<<"\n\n🚀 TOTAL GPU EXECUTION TIME = "<<ms/1000<<" seconds\n";
}


Writing loa_gpu.cu


In [2]:
!nvcc -arch=sm_75 loa_gpu.cu -o loa_gpu.out


In [3]:
!./loa_gpu.out

Iteration 1000 | best = 0
Iteration 2000 | best = 0
Iteration 3000 | best = 0
Iteration 4000 | best = 0
Iteration 5000 | best = 0
Iteration 6000 | best = 0
Iteration 7000 | best = 0
Iteration 8000 | best = 0

🔥 FINAL BEST = 0
Best Solution = 1.78958e-23 -5.77204e-24 5.78714e-26 -2.53587e-23 2.16519e-23 

🚀 TOTAL GPU EXECUTION TIME = 0.281461 seconds


In [5]:
%%writefile seq_loa.cpp

#include <bits/stdc++.h>
#include<omp.h>

using namespace std;

// Random generator
mt19937 rng(time(NULL));
double randF(double a, double b)
{
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}

// ------------------------------
// Sphere Function
// ------------------------------
double sphere(const vector<double> &x)
{
    double s = 0;
    for (double v : x)
        s += v * v;
    return s;
}

// ------------------------------
// LOA PARAMETERS
// ------------------------------
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

// ------------------------------
// Initialize population
// ------------------------------
vector<vector<double>> init_population()
{
    vector<vector<double>> pop(POP, vector<double>(DIM));
    for (int i = 0; i < POP; i++)
        for (int d = 0; d < DIM; d++)
            pop[i][d] = randF(LB, UB);
    return pop;
}

// ------------------------------
// Escape move (global search)
// ------------------------------
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA)
{
    int DIM = x.size();
    vector<double> newX(DIM);

    for (int j = 0; j < DIM; j++)
    {
        double r = randF(0, 1);
        int I = (rng() % 2) + 1; // I = 1 or 2

        newX[j] = x[j] + r * (SSA[j] - I * x[j]);

        // Bound check
        if (newX[j] < LB)
            newX[j] = LB;
        if (newX[j] > UB)
            newX[j] = UB;
    }

    return newX;
}

// ------------------------------
// Hide move (local search)
// ------------------------------
vector<double> hide(const vector<double> &Xi, int t) {
    vector<double> newX = Xi;

    for (int j = 0; j < DIM; j++) {
        double r = randF(0, 1);

        newX[j] = Xi[j] + (1 - 2*r) * (UB - LB) / t;

        // bounds
        if (newX[j] < LB) newX[j] = LB;
        if (newX[j] > UB) newX[j] = UB;
    }
    return newX;
}

// ------------------------------
// MAIN LOA ALGORITHM
// ------------------------------
int main()
{
    auto t_start = chrono::high_resolution_clock::now();

    auto pop = init_population(); // random solution create kia idhar
    vector<double> fitness(POP);  // sabhi lyrebird ki fitness store krne k liye

    // Evaluate initial fitness
    for (int i = 0; i < POP; i++)
        fitness[i] = sphere(pop[i]); // initial fitness calculate kr liya

    // Track best
    double bestFit = 1e18; // initial best fitness ko bahut bada rkh dia

    vector<double> bestSol(DIM);

    for (int i = 0; i < POP; i++)
    {
        if (fitness[i] < bestFit)
        {
            bestFit = fitness[i];
            bestSol = pop[i];
        }
    }

    // -------------------------------------
    // LOA main loop
    // -------------------------------------

    for (int it = 1; it <= MAX_IT; it++)
    {

        for (int i = 0; i < POP; i++)
        {

            // Pick random better candidate
            int betterIdx = -1;

            vector<int> betterList;
            for (int j = 0; j < POP; j++)
                if (fitness[j] < fitness[i])
                    betterList.push_back(j);

            if (!betterList.empty())
                betterIdx = betterList[rng() % betterList.size()];

            // Choose escape or hide
            vector<double> candidate;
            double rp = randF(0, 1);

            if (rp < 0.5 && betterIdx != -1)
                candidate = escape(pop[i], pop[betterIdx]);
            else
                candidate = hide(pop[i], it);

            double f = sphere(candidate);

            // Accept if better
            if (f < fitness[i])
            {
                pop[i] = candidate;
                fitness[i] = f;

                if (f < bestFit)
                {
                    bestFit = f;
                    bestSol = candidate;
                }
            }
        }

        // OPTIONAL: Print progress
        cout << "Iteration " << it
             << " | Best = " << bestFit << endl;
    }

    // -------------------------------------
    // Final Output
    // -------------------------------------
    cout << "\nBest Solution Found:\n";
    for (int d = 0; d < DIM; d++)
        cout << "x" << d + 1 << " = " << bestSol[d] << endl;

    cout << "\nBest Sphere Value = " << bestFit << endl;

    auto t_end = chrono::high_resolution_clock::now();
    double elapsed = chrono::duration_cast<chrono::duration<double>>(t_end - t_start).count();
    cout << "Total execution time: " << elapsed << " seconds" << endl;

    return 0;
}



Writing seq_loa.cpp


In [6]:
!g++ seq_loa.cpp -o sol
!./sol

Streaming output truncated to the last 5000 lines.
Iteration 3011 | Best = 0
Iteration 3012 | Best = 0
Iteration 3013 | Best = 0
Iteration 3014 | Best = 0
Iteration 3015 | Best = 0
Iteration 3016 | Best = 0
Iteration 3017 | Best = 0
Iteration 3018 | Best = 0
Iteration 3019 | Best = 0
Iteration 3020 | Best = 0
Iteration 3021 | Best = 0
Iteration 3022 | Best = 0
Iteration 3023 | Best = 0
Iteration 3024 | Best = 0
Iteration 3025 | Best = 0
Iteration 3026 | Best = 0
Iteration 3027 | Best = 0
Iteration 3028 | Best = 0
Iteration 3029 | Best = 0
Iteration 3030 | Best = 0
Iteration 3031 | Best = 0
Iteration 3032 | Best = 0
Iteration 3033 | Best = 0
Iteration 3034 | Best = 0
Iteration 3035 | Best = 0
Iteration 3036 | Best = 0
Iteration 3037 | Best = 0
Iteration 3038 | Best = 0
Iteration 3039 | Best = 0
Iteration 3040 | Best = 0
Iteration 3041 | Best = 0
Iteration 3042 | Best = 0
Iteration 3043 | Best = 0
Iteration 3044 | Best = 0
Iteration 3045 | Best = 0
Iteration 3046 | Best = 0
Iteration 304

In [3]:
%%writefile omp_loa.cpp

#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

/* ---------------------------------------
      Thread-safe Random Generator
---------------------------------------*/
double randF(mt19937 &rng, double a, double b) {
    uniform_real_distribution<double> dist(a, b);
    return dist(rng);
}
int randInt(mt19937 &rng, int a, int b) {
    uniform_int_distribution<int> dist(a, b);
    return dist(rng);
}

/* ---------------------------------------
            Sphere Function
---------------------------------------*/
double sphere(const vector<double> &x){
    double s = 0;
    for(double v : x)  s += v*v;
    return s;
}

/* ---------------------------------------
            LOA PARAMETERS
---------------------------------------*/
int POP = 256;
int DIM = 5;
int MAX_IT = 8000;
double LB = -100;
double UB = 100;

/* ---------------------------------------
     Escape (Global Search)  — parallel safe
---------------------------------------*/
vector<double> escape(const vector<double> &x,
                      const vector<double> &SSA,
                      mt19937 &rng)
{
    vector<double> newX = x;
    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        int I = randInt(rng,1,2);
        newX[j] = x[j] + r * (SSA[j] - I*x[j]);

        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
     Hide (Local Search) — parallel safe
---------------------------------------*/
vector<double> hide(const vector<double> &Xi,int t,mt19937 &rng){
    vector<double> newX = Xi;

    for(int j=0;j<DIM;j++){
        double r = randF(rng,0,1);
        newX[j] = Xi[j] + (1 - 2*r)*(UB-LB)/t;
        newX[j] = min(max(newX[j],LB),UB);
    }
    return newX;
}

/* ---------------------------------------
         Initialize population (PARALLEL)
---------------------------------------*/
vector<vector<double>> init_population(vector<mt19937> &rngs){
    vector<vector<double>> pop(POP, vector<double>(DIM));

    #pragma omp parallel
    {
        int tid = omp_get_thread_num();
        mt19937 &local_rng = rngs[tid];

        #pragma omp for schedule(static)
        for(int i=0;i<POP;i++)
            for(int d=0;d<DIM;d++)
                pop[i][d] = randF(local_rng,LB,UB);
    }
    return pop;
}

/* =======================================
             MAIN LOA PARALLEL
=======================================*/
int main(){
    auto t1 = chrono::high_resolution_clock::now();

    int threads = omp_get_max_threads();
    vector<mt19937> rngs(threads);

    random_device rd;
    for(int i=0;i<threads;i++)
        rngs[i].seed(rd()+i*111);

    vector<vector<double>> pop = init_population(rngs);
    vector<double> fitness(POP);

    // Initial fitness
    for(int i=0;i<POP;i++) fitness[i]=sphere(pop[i]);

    double bestFit = 1e18;
    vector<double> bestSol(DIM);

    // find initial best
    for(int i=0;i<POP;i++){
        if(fitness[i]<bestFit){
            bestFit=fitness[i];
            bestSol=pop[i];
        }
    }

    /* --------------------------------------
                LOA ITERATIONS
       Full population parallel every step
    ---------------------------------------*/
    for(int it=1; it<=MAX_IT; it++){

        #pragma omp parallel
        {
            int tid = omp_get_thread_num();
            mt19937 &localRng = rngs[tid];

            double localBest = 1e18;
            vector<double> localBestSol(DIM);

            #pragma omp for schedule(static)
            for(int i=0;i<POP;i++){

                vector<int> better;
                for(int j=0;j<POP;j++)
                    if(fitness[j]<fitness[i]) better.push_back(j);

                int betterIdx=-1;
                if(!better.empty())
                    betterIdx = better[randInt(localRng,0,(int)better.size()-1)];

                vector<double> candidate;
                if(randF(localRng,0,1)<0.5 && betterIdx!=-1)
                    candidate = escape(pop[i],pop[betterIdx],localRng);
                else
                    candidate = hide(pop[i],it,localRng);

                double f = sphere(candidate);

                if(f<fitness[i]){
                    pop[i]=candidate;
                    fitness[i]=f;
                }
                if(f<localBest){
                    localBest=f;
                    localBestSol=candidate;
                }
            }

            // Update global best safely
            #pragma omp critical
            {
                if(localBest<bestFit){
                    bestFit=localBest;
                    bestSol=localBestSol;
                }
            }
        }

        cout<<"Iter "<<it<<" | Best = "<<bestFit<<"\n";
    }

    cout<<"\nFinal Best Solution:\n";
    for(int i=0;i<DIM;i++) cout<<"x"<<i+1<<" = "<<bestSol[i]<<endl;
    cout<<"\nBest Sphere Value = "<<bestFit<<endl;

    auto t2 = chrono::high_resolution_clock::now();
    cout<<"\nExecution Time = "
        <<chrono::duration<double>(t2-t1).count()
        <<" sec\n";

    return 0;
}


Overwriting omp_loa.cpp


In [4]:
!g++ -fopenmp omp_loa.cpp -o sol
!./sol

Streaming output truncated to the last 5000 lines.
Iter 3012 | Best = 0
Iter 3013 | Best = 0
Iter 3014 | Best = 0
Iter 3015 | Best = 0
Iter 3016 | Best = 0
Iter 3017 | Best = 0
Iter 3018 | Best = 0
Iter 3019 | Best = 0
Iter 3020 | Best = 0
Iter 3021 | Best = 0
Iter 3022 | Best = 0
Iter 3023 | Best = 0
Iter 3024 | Best = 0
Iter 3025 | Best = 0
Iter 3026 | Best = 0
Iter 3027 | Best = 0
Iter 3028 | Best = 0
Iter 3029 | Best = 0
Iter 3030 | Best = 0
Iter 3031 | Best = 0
Iter 3032 | Best = 0
Iter 3033 | Best = 0
Iter 3034 | Best = 0
Iter 3035 | Best = 0
Iter 3036 | Best = 0
Iter 3037 | Best = 0
Iter 3038 | Best = 0
Iter 3039 | Best = 0
Iter 3040 | Best = 0
Iter 3041 | Best = 0
Iter 3042 | Best = 0
Iter 3043 | Best = 0
Iter 3044 | Best = 0
Iter 3045 | Best = 0
Iter 3046 | Best = 0
Iter 3047 | Best = 0
Iter 3048 | Best = 0
Iter 3049 | Best = 0
Iter 3050 | Best = 0
Iter 3051 | Best = 0
Iter 3052 | Best = 0
Iter 3053 | Best = 0
Iter 3054 | Best = 0
Iter 3055 | Best = 0
Iter 3056 | Best = 0
Iter